In [1]:
cd /home/jovyan

/home/jovyan


In [2]:
from lib.database import query_database, query_to_dataframe 

In [3]:
query_to_dataframe("""
    SELECT COUNT(*) FROM adult WHERE (
        (native_country = ' ?') OR
        (occupation = ' ?') OR
        (workclass = ' ?')
    )
""")

,count
0,2399


In [4]:
pre_etl_adult_df = query_to_dataframe("""
    SELECT
        age,
        capital_gain,
        capital_loss,
        education,
        gender,
        hours_per_week, 
        income_label,
        marital_status,
        native_country,
        occupation,
        race,
        relationship,
        workclass
    FROM adult
    WHERE (
        (native_country != ' ?') AND
        (occupation != ' ?') AND
        (workclass != ' ?')
    );
""")

In [5]:
pre_etl_adult_df.shape

(30162, 13)

In [6]:
pre_etl_adult_df.head()

,age,capital_gain,capital_loss,education,gender,hours_per_week,income_label,marital_status,native_country,occupation,race,relationship,workclass
0,39,2174,0,Bachelors,Male,40,<=50K,Never-married,United-States,Adm-clerical,White,Not-in-family,State-gov
1,50,0,0,Bachelors,Male,13,<=50K,Married-civ-spouse,United-States,Exec-managerial,White,Husband,Self-emp-not-inc
2,38,0,0,HS-grad,Male,40,<=50K,Divorced,United-States,Handlers-cleaners,White,Not-in-family,Private
3,53,0,0,11th,Male,40,<=50K,Married-civ-spouse,United-States,Handlers-cleaners,Black,Husband,Private
4,28,0,0,Bachelors,Female,40,<=50K,Married-civ-spouse,Cuba,Prof-specialty,Black,Wife,Private


In [7]:
pre_etl_adult_df.dtypes

age                int64
capital_gain       int64
capital_loss       int64
education         object
gender            object
hours_per_week     int64
income_label      object
marital_status    object
native_country    object
occupation        object
race              object
relationship      object
workclass         object
dtype: object

In [8]:
pre_etl_adult_df.income_label.unique(), pre_etl_adult_df.gender.unique()

(array([' <=50K', ' >50K'], dtype=object),
 array([' Male', ' Female'], dtype=object))

In [9]:
pre_etl_adult_df["income_label"] = (pre_etl_adult_df["income_label"] == " >50K").astype(int)
pre_etl_adult_df["gender"] = (pre_etl_adult_df["gender"] == " Female").astype(int)

In [10]:
from psycopg2 import ProgrammingError
try:
    query_database("""
    BEGIN;
    DROP TABLE adult_for_modeling;
    COMMIT;
    """)
except ProgrammingError as e:
    print(e)

query_database("""
    BEGIN;
    CREATE TABLE adult_for_modeling (
        age INTEGER,
        capital_gain INTEGER,
        capital_loss INTEGER,
        education TEXT,
        gender INTEGER,
        hours_per_week INTEGER,
        income_label INTEGER,
        marital_status TEXT,
        native_country TEXT,
        occupation TEXT,
        race TEXT,
        relationship TEXT,
        workclass TEXT
    );
    COMMIT;
    """, fetch_res=False)
print("adult_for_modeling table created")

no results to fetch
adult_for_modeling table created


In [11]:
query_database(f"""
    BEGIN;
    INSERT INTO adult_for_modeling (
        age,
        capital_gain,
        capital_loss,
        education,
        gender,
        hours_per_week,
        income_label,
        marital_status,
        native_country,
        occupation,
        race,
        relationship,
        workclass
    )
    VALUES {','.join(str(record) for record in pre_etl_adult_df.to_records(index=False))};
    COMMIT;
""", fetch_res=False)

In [12]:
query_database("SELECT COUNT(*) FROM adult_for_modeling")

[{'count': 30162}]

In [13]:
from psycopg2 import ProgrammingError
try:
    query_database("""
    BEGIN;
    DROP TABLE adult_for_modeling;
    COMMIT;
    """)
except ProgrammingError as e:
    print(e)

no results to fetch


In [14]:
%run src/load_data.py

table "adult_for_modeling" does not exist

adult_for_modeling table created


In [15]:
query_database("SELECT COUNT(*) FROM adult_for_modeling")

[{'count': 30162}]